In [69]:
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
from random import randrange

F = Zmod(0x1fffffffffe00001)

def gaussian(rows, cols:int, sigma:float):
    X = DiscreteGaussianDistributionIntegerSampler(sigma)
    return matrix(F, rows, cols, lambda i, j: X())


l = 16
d = 16
u =  Matrix([[1 for i in range(d)]])
witness = [F(randrange(0, 2)) for i in range(l)]
sigma = 3.2

M = [[0, 0] for i in range(l)]
for i in range(l):
    M[i][0] = matrix(F, d, d, lambda i,j: F(randrange(0, 2)))
    M[i][1] = matrix(F, d, d, lambda i,j: F(randrange(0, 2)))


def Gen(u, M, x):
    
    n = len(x)
    
    A = 0
    D = [[0, 0] for i in range(n-1)]
    
    print(n)
    
    # A1 = ((M2 x S2)*A2 + E)*D2^-1
    # A0 = (((M1 x S1)*A1 + E)*D1^-1 + E)
    out = 0
    
    for i in range(n-1, 0, -1):
        
        if i == n-1:
            A = matrix(F, d*d, d*d, lambda i, j: F.random_element())
            out = A
            
        S = matrix(F, d, d, lambda i,j: F.random_element(2))

        D[i-1][0] = gaussian(d*d, d*d, sigma)
        D[i-1][1] = gaussian(d*d, d*d, sigma)
        
        out = (M[i][x[i]].tensor_product(S) * out + gaussian(d*d, d*d, 3.2)) * D[i-1][x[i]]^-1
            
        if i == n-1:
            S_witness = S
        else:
            S_witness = S * S_witness
        
    # C0 = (uM0 x S0) * A0 + E
    C = [0, 0]
    S = matrix(F, d, d, lambda i,j: F.random_element(2))
    
    if x[0] == 0:
        C[0] = (u * M[0][0]).tensor_product(S) * out + gaussian(d, d*d, 3.2)
        C[1] = matrix(F, d*d, d*d, lambda i, j: F.random_element())
    else:
        C[0] = matrix(F, d*d, d*d, lambda i, j: F.random_element())
        C[1] = (u * M[0][1]).tensor_product(S) * out + gaussian(d, d*d, 3.2)
        
    S_witness = S * S_witness
    
    CHECK = u
    for i in range(n):
        CHECK *= M[i][x[i]]
    CHECK = CHECK.tensor_product(S_witness) * A

    return C, D, CHECK

def Eval(C, D, CHECK, witness):
    
    OUT = C[witness[0]]
    for i in range(len(witness)-1):
        OUT *= D[i][witness[i+1]]
    return OUT - CHECK

C, D, CHECK = Gen(u, M, witness)

print(CHECK)

print()
print(Eval(C, D, CHECK, witness))

#Example for l = 3
# C0 * D1 * D2
# = ((uM0 x S0) * A0 + E) * D1 * D2
# = ((uM0 x S0) * ((M1 x S1)*A1 + E)*D1^-1 + E) * D1 * D2
# = ((uM0 x S0) * ((M1 x S1)*A1 + E) + E*D1) * D2
# = ((uM0 x S0) * ((M1 x S1)*((M2 x S2)*A2 + E)*D2^-1 + E) + E*D1) * D2
# = ((uM0 x S0) * ((M1 x S1)*((M2 x S2)*A2 + E) + E*D2) + E*D1*D2)
# = ((uM0 x S0) * (M1 x S1) * (M2 x S2) * A2 + [E + E*D2 + E*D1*D2]
# = ((uM0 * M1 * M2) x (S0 * S1 * S2))*A2 + E




16
[1297469287845849137 1226585832775991828 1230676616332462959 1857174302248131476  466669417135682475  695491655574434968 1254300290789471425  880357208001267849 1171471245624569184 1299123246510906270 1958894088971810539 1461768402772587951  704437482257433790 1076300741636387670 1107167042147517975 1305449694600231078 1264689361721841495  546632670030542408 2186788623388574180 1336205823859829438  718372974849320214  612270591320587908 1738984512349171057 1447161136630932476 2118577228892528834  257952473916574686 1563031348693924873 1161083333894451968  231041399402258297 1901428737309264328 1857313316722400573  807245526076556533 2071701716745632154  162676514523290197 2000078344855013744  517701154804499194  101309265167171146 1453589295831037388  955674048417678612  139433266428471404  610755221476446230 2100082710341423983 1717328196496638436  546606329355442134  218281793599756765 1201884871002886207 1218622580864477655 1202282533004298762 1282419943020800141  395703058788903

[1745998600761125244 2020820520001778129  280616307949359344  738568317179825467  935916287964795933 1671704644542307295 1746342720730781898 1570341323106983227 1629788489374582817 1904027885521044628  781210571228285573  524923874870855906 1666530874971075737  261479888822897015  494802674186290409 1673897392827542181  642060934552887496 2281983602051113147 2238408442839198814 1143205945362208518   45991053824429597 1528968112651936092  665121633407988505 2256037863505737720 1844660307612391714 1898077900571898565  698875355999360931 1154636619696884215 2107947239125845503 1973289770990449207  185690121129670043 1621694819859064536 1169565202488556877 1281307439601110053  389549963357186096 1779288232491768375  187816512495516934  113193574701737372 1769964847645984947 1702229895594376531  814262080713717922  327573032972840497 1539253814556564194 2084642403727662065  775221389238506671  112112746720202913 1555380385483623007  961227438263049246  970359085009078423  913425468942948328